In [ ]:
pip install wandb datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.2/213.2 kB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.3 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=177dd9a3345152100075191a67bf4199b91b7ed244dc888c67cb402710b70ce6
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8

In [ ]:
pip install git+https://github.com/huggingface/accelerate

  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-qtm_1uap
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-qtm_1uap
  Resolved https://github.com/huggingface/accelerate to commit 180ef7c41576bc55cb1265fcdeb7da287e95c9b7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.21.0.dev0-py3-none-any.whl size=241741 sha256=6dff2f73c6158b613c2d7ea5bf1e30eb1f68e83c14885c0d65c0ddb438e01c61
  Stored in directory: /tmp/pip-ephem-wheel-cache-hz8le0e4/wheels/f6/c7/9d/1b8a5ca8353d9307733bc719107acb67acdc95063bba749f26
Successfully built accelerate


In [ ]:
pip install transformers==4.28.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.7 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    MBartForSequenceClassification,
    TrainingArguments,
    Trainer,
)
from tqdm import tqdm

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict

# Read the CSV dataframe
df = pd.read_csv("Data/corrected_data.csv")

df = df.sample(frac=1, random_state=42)

# df = df.sample(frac=0.1)
# Split the dataframe into train, val, and test splits
train_df = df.sample(frac=0.8)
val_df = df.drop(train_df.index)
test_df = val_df.sample(frac=0.5)

# Create the hugging face dataset
dataset = DatasetDict()
dataset["train"] = Dataset.from_pandas(train_df)
dataset["val"] = Dataset.from_pandas(val_df)
dataset["test"] = Dataset.from_pandas(test_df)

In [ ]:
model_name = "facebook/mbart-large-cc25"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def preprocess(examples):
    return tokenizer(
        examples["Content"], truncation=True, padding="max_length", max_length=128
    )

In [ ]:
dataset = dataset.map(preprocess, batched=True)

Map:   0%|          | 0/30428 [00:00<?, ? examples/s]

Map:   0%|          | 0/7607 [00:00<?, ? examples/s]

Map:   0%|          | 0/3804 [00:00<?, ? examples/s]

In [ ]:
import wandb
import numpy as np
import torch

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['Content', 'labels', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 30428
    })
    val: Dataset({
        features: ['Content', 'labels', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 7607
    })
    test: Dataset({
        features: ['Content', 'labels', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 3804
    })
})


In [ ]:
model = MBartForSequenceClassification.from_pretrained("facebook/mbart-large-cc25", num_labels=2)

Some weights of the model checkpoint at facebook/mbart-large-cc25 were not used when initializing MBartForSequenceClassification: ['final_logits_bias']
- This IS expected if you are initializing MBartForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MBartForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MBartForSequenceClassification were not initialized from the model checkpoint at facebook/mbart-large-cc25 and are newly initialized: ['classification_head.out_proj.weight', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.dense.bias']
You should probably TRAIN this model on a down-stream ta

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

def objective(model, weight_decay, num_train_epochs, train_batch, eval_batch, optimizer, lr_scheduler):



#     device = torch.device('cuda')
#     model.to(device)
#     training_args = TrainingArguments(
#         output_dir="mBart-exp2/",
#         weight_decay=weight_decay,
#         num_train_epochs=num_train_epochs,
#         per_device_train_batch_size=train_batch,
#         per_device_eval_batch_size=eval_batch,
#         gradient_accumulation_steps = 4,
# #         disable_tqdm=True,
#         report_to = 'wandb'
#     )
#     trainer = Trainer(
#         model=model,
#         args=training_args,
#         train_dataset=dataset["train"],
#         eval_dataset=dataset["val"],
#         optimizers = (optimizer,lr_scheduler)
#     )

#     wandb.init(project="Unmasking-Hate", name="mBart-tel-run-custom-2")

#     trainer.train()

    # Evaluate the model after each epoch.
    progress_bar = tqdm(range(training_args.num_train_epochs), desc="Epochs", position=0)
    for epoch in progress_bar:
        train_loss = 0.0
        train_predictions = []
        train_targets = []
        val_predictions = []
        val_targets = []
        for batch in trainer.get_eval_dataloader():
        #     model_input = batch["input_ids"].to(trainer.model.device)
        #     targets = batch["labels"].to(trainer.model.device)

        #     outputs = trainer.model(model_input, labels=targets)
        #     loss = outputs.loss
        #     logits = outputs.logits

        #     train_loss += loss.item()
        #     train_predictions.append(logits.detach().cpu().numpy())
        #     train_targets.append(targets.detach().cpu().numpy())

        # train_loss /= len(trainer.get_eval_dataloader())
        # train_predictions = np.concatenate(train_predictions, axis=0)
        # train_targets = np.concatenate(train_targets, axis=0)
        # train_accuracy = (train_predictions.argmax(axis=1) == train_targets).mean()

        val_loss = trainer.evaluate().get("eval_loss", None)

        # Call trainer.predict() on the val dataset
        val_outputs = trainer.predict(dataset["val"])
        val_predictions = val_outputs.predictions
        val_targets = val_outputs.label_ids

        val_accuracy = (val_predictions.argmax(axis=1) == val_targets).mean()

        precision, recall, f1, _ = precision_recall_fscore_support(
            val_targets, val_predictions.argmax(axis=1), average="weighted"
        )

        print(f"Epoch: {epoch + 1}",f"Train Loss: {train_loss}",f"Val Loss: {val_loss}",f"Train Accuracy: {train_accuracy}",f"Val Accuracy: {val_accuracy}",f"Precision: {precision}",f"Recall: {recall}",f"F1: {f1}")

        wandb.log(
            {
                "Epoch": epoch + 1,
                "Train Loss": train_loss,
                "Val Loss": val_loss,
                "Train Accuracy": train_accuracy,
                "Val Accuracy": val_accuracy,
                "Precision": precision,
                "Recall": recall,
                "F1": f1,
            }
        )

    wandb.finish()

    return


IndentationError: ignored

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from transformers import AdamW
from transformers import get_cosine_schedule_with_warmup
optimizer = AdamW(model.parameters(), lr=1e-3)
num_training_steps = 10
lr_scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=0.1*num_training_steps, num_training_steps=num_training_steps)
objective(model,0.0001,3,4,4, optimizer, lr_scheduler)

NameError: ignored

In [ ]:
model.save_pretrained('mBart_exp2/hf/')

In [ ]:
# Save the model weights
torch.save(model.state_dict(), "mBart_exp2/mBart-model.pt")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')